In [ ]:
from mdl.Flight import Boid2D
from mdl.Graphics import Graphics
from moviepy.editor import VideoClip
import numpy as np
import math

In [ ]:
def add_lift(boid, dt=1):
    # Calculate lift
    lift_force = np.array(boid.lift)

    delta_V = (dt * lift_force) / boid.Mass


    newVx = boid.Vx + delta_V[0]
    newVy = boid.Vy + delta_V[1]
    newV = np.sqrt(newVx ** 2 + newVy **2)
    scale = boid.V / newV
    boid.Vx = newVx * scale
    boid.Vy = newVy * scale


def update(boid, dt=1):
    add_lift(boid, dt = dt)
    # Calculate forces
    drag_force = np.array(boid.drag)
    gravity_force = np.array(boid.gravity)

    # Net force
    net_force = gravity_force + drag_force 

    acceleration = net_force / boid.Mass
    boid.Vx += acceleration[0] * dt
    boid.Vy += acceleration[1] * dt

    # Update position
    boid.X += boid.Vx * dt
    boid.Y += boid.Vy * dt

    boid.atmosphere.height = boid.Y
    
def fastupdate(boid, dt=1):
    # Calculate forces
    drag_force = np.array(boid.drag)
    lift_force = np.array(boid.lift)
    gravity_force = np.array(boid.gravity)

    # Net force
    net_force = gravity_force + drag_force + lift_force

    acceleration = net_force / boid.Mass
    boid.Vx += acceleration[0] * dt
    boid.Vy += acceleration[1] * dt


    # Update position
    boid.X += boid.Vx * dt
    boid.Y += boid.Vy * dt


    boid.atmosphere.height = boid.Y


In [ ]:
boid = Boid2D(Y = 150, X = 50, Vx = 15) 

PixelToMeter = 0.5

window_width = 800
window_height = 600

graphics = Graphics(
    width = window_width,
    height= window_height,
    x = 2300,
    y = 200
)

graphics.save_on_quit('../amt/singleboid.gif', fps=60)

graphics.init()

running = True

while running:
    for event in graphics.event.get():
        if event.type == graphics.QUIT:
            running = False
        if event.type == graphics.KEYDOWN:
            if event.key == graphics.K_SPACE:
               boid.flap()

    update(boid, dt = 0.016)

    graphics.blank()

    position = (boid.X / PixelToMeter, (boid.Y / PixelToMeter))

    graphics.draw_vector(position = position, vector = (boid.Vx, boid.Vy), color = (255,0,0))
    graphics.draw_vector(position = position, vector = boid.drag, color = (255,0,0))
    graphics.draw_vector(position = position, vector = boid.lift, color = (0,255,0))
    graphics.draw_vector(position = position, vector = boid.gravity, color = (0,0,0))
    graphics.draw_boid(  position = position, angle = math.atan2(boid.Vy,boid.Vx), color = (0,0,255) if boid.can_flap() else (255,0,0))
    graphics.draw_stats(stats = [
        f"V: {int(boid.V)} m/s Vx: {int(boid.Vx)} m/s Vy: {int(boid.Vy)} m/s",
        f"E: {int(boid.energy)} J",
        f"H: {int(boid.Y)} m"
    ])
    
    graphics.display()

    graphics.clock.tick(60)

graphics.quit()


In [ ]:
import os
import math
import pygame
from moviepy.editor import VideoClip, ImageSequenceClip
import numpy as np
self = graphics
fps = 10
filename = '../amt/singleboid.gif'
with VideoClip(lambda t: self.frames[int(t*fps)], duration=len(self.frames) // fps) as clip:
                if filename.endswith('.gif'):
                    clip.write_gif(filename, fps=fps)